In [2]:
from igo.Tagger import Tagger
t = Tagger() # use bundled dictionary
for m in t.parse(u'すもももももももものうち'):
    print(m.surface, m.feature)

すもも 名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も 助詞,係助詞,*,*,*,*,も,モ,モ
もも 名詞,一般,*,*,*,*,もも,モモ,モモ
も 助詞,係助詞,*,*,*,*,も,モ,モ
もも 名詞,一般,*,*,*,*,もも,モモ,モモ
の 助詞,連体化,*,*,*,*,の,ノ,ノ
うち 名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ


In [9]:
from igo.Tagger import Tagger
sent='このたんごのいみをおぼえてください'

t = Tagger() # use bundled dictionary
for m in t.parse(sent):
    print(m.surface, m.feature)

この 連体詞,*,*,*,*,*,この,コノ,コノ
たん 名詞,一般,*,*,*,*,たん,タン,タン
ご 接頭詞,名詞接続,*,*,*,*,ご,ゴ,ゴ
の 助詞,連体化,*,*,*,*,の,ノ,ノ
いみ 動詞,自立,*,*,五段・マ行,連用形,いむ,イミ,イミ
を 助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
おぼえ 動詞,自立,*,*,一段,連用形,おぼえる,オボエ,オボエ
て 助詞,接続助詞,*,*,*,*,て,テ,テ
ください 動詞,非自立,*,*,五段・ラ行特殊,命令ｉ,くださる,クダサイ,クダサイ


In [1]:
import tinysegmenter
statement = '私はpython大好きStanding Engineerです．'
tokenized_statement = tinysegmenter.tokenize(statement)
print(tokenized_statement)

['私', 'は', 'python', '大好き', 'Standing', ' ', 'Engineer', 'です', '．']


In [10]:
import igo.Tagger
import sagas.ja.whoosh
from sagas.ja.whoosh.IgoTokenizer import IgoTokenizer
from sagas.ja.whoosh.TinySegmenterTokenizer import TinySegmenterTokenizer
from sagas.ja.whoosh.MeCabTokenizer import MeCabTokenizer

# tk = IgoTokenizer()
# tk = TinySegmenterTokenizer()
tk = MeCabTokenizer()

In [16]:
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser
import os
import shutil

target_dir="out/indexdir"
schema = Schema(title=TEXT(stored=True), 
                path=ID(stored=True), 
                # content=TEXT)
                content=TEXT(analyzer=tk))

if os.path.exists(target_dir):
    shutil.rmtree(target_dir)
os.mkdir(target_dir)
ix = create_in(target_dir, schema)
writer = ix.writer()

writer.add_document(title=u"First document", path=u"/a",
                    content=u"このたんごのいみをおぼえてください")
writer.add_document(title=u"Second document", path=u"/b",
                    content=u"彼女を呼びます。")
writer.commit()

In [17]:
def search_word(word):
    with ix.searcher() as searcher:
        query = QueryParser("content", ix.schema).parse(word)
        results = searcher.search(query)
        print(results[0])

search_word("おぼえ")

<Hit {'path': '/a', 'title': 'First document'}>


In [18]:
search_word("呼び")

<Hit {'path': '/b', 'title': 'Second document'}>


In [23]:
import whoosh.qparser as qparser
word="second"
with ix.searcher() as searcher:
    parser = qparser.QueryParser("title", ix.schema)
    parser.add_plugin(qparser.FuzzyTermPlugin())
    query=parser.parse(word)
    results = searcher.search(query)
    print(results[0])

<Hit {'path': '/b', 'title': 'Second document'}>


In [24]:
print(query)

title:second
